## This work is going to be all based on Ken Jee work about the titanic course. I'm using his youtube video and his kaggle notebook to learn Data Science. 

Video: https://www.youtube.com/watchv=I3FBJdiExcg&t=581s&ab_channel=KenJeeKenJee

Notebook: https://www.kaggle.com/kenjee/titanic-project-example

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
training = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

training['train_test'] = 1
test['train_test'] = 0
test['Survived'] = np.NaN
all_data = pd.concat([training, test])

%matplotlib inline
all_data.columns

## Light data exploration

### 1) For numeric data
- Made histograms to understand distributions
- Corrplot
- Pivot table comparing survival rate across numeric values

### 2) For categorical data
- Made bar charts tounderstand balance of classes
- Made pivot tables to understand relationship with survival

In [ ]:
#quick look at our dat types & null counts

training.info()

In [ ]:
# use describe to understand better the data
training.describe()

In [ ]:
#we are going to didivde values divided in numeric and categorical
df_num = training[['Age', 'SibSp', 'Parch', 'Fare']]
df_cat = training[['Survived', 'Pclass', 'Sex', 'Ticket', 'Cabin', 'Embarked']]


In [ ]:
#see distribution of Age
plt.hist(df_num['Age'])
plt.title('Age')
plt.show

In [ ]:
#see distribution of SibSp
plt.hist(df_num['SibSp'])
plt.title('SibSp')
plt.show

In [ ]:
#see distribution of Parch
plt.hist(df_num['Parch'])
plt.title('Parch')
plt.show

In [ ]:
#see distribution of Fare
plt.hist(df_num['Fare'])
plt.title('Fare')
plt.show

#### What would happen if we tried normalizing the non normal distributions? (Fare, Parch and SibSp)

In [ ]:
print(df_num.corr())
sns.heatmap(df_num.corr())

Now we want to look at how survival rates differ across this different groups

In [ ]:
#compare survival rates across Age, SibSp, Parch and Fare
#pd.pivot_table(training, index = 'Survived', values = ['Age', 'SibSp', 'Parch', 'Fare'])

### Now we'll look at graphs for categorical data

In [ ]:
#Survived
sns.barplot(df_cat['Survived'].value_counts().index, df_cat['Survived'].value_counts()).set_title('Survived')
plt.show

In [ ]:
#Pclass
sns.barplot(df_cat['Pclass'].value_counts().index, df_cat['Pclass'].value_counts()).set_title('Pclass')
plt.show

In [ ]:
#Sex
sns.barplot(df_cat['Sex'].value_counts().index, df_cat['Sex'].value_counts()).set_title('Sex')
plt.show

In [ ]:
#Ticket
sns.barplot(df_cat['Ticket'].value_counts().index, df_cat['Ticket'].value_counts()).set_title('Ticket')
plt.show

In [ ]:
#Cabin
sns.barplot(df_cat['Cabin'].value_counts().index, df_cat['Cabin'].value_counts()).set_title('Cabin')
plt.show

In [ ]:
#Embarked
sns.barplot(df_cat['Embarked'].value_counts().index, df_cat['Embarked'].value_counts()).set_title('Embarked')
plt.show

### Now I'll want to compare the categorical variables to see if there's correlation between the people who survived and the different categories of each variable

In [ ]:
pd.pivot_table(training, index = 'Survived', columns='Pclass', values='Ticket', aggfunc = 'count')

We can conclude that people from higher classes, survived more than people from lower classes

In [ ]:
pd.pivot_table(training, index = 'Survived', columns='Sex', values='Ticket', aggfunc = 'count')

In [ ]:
print('Percentage of females who survived: ', 233/(81+233))
print('Percentage of males who survived: ', 109/(109+468))

Here, we can say that only 18% of males survived, whether 74% of females survived.

In [ ]:
pd.pivot_table(training, index = 'Survived', columns='Embarked', values='Ticket', aggfunc = 'count')

In [ ]:
print('Percentage of people who embarked in Cherbourgh who survived: ', 93/(93+75))
print('Percentage of people who embarked in Queenstown who survived: ', 30/(30+47))
print('Percentage of people who embarked in Southhampton who survived: ', 217/(427+217))

People who embarked in cherbourgh are more likely to survive, then queenstown and at last, people who survived the least embarked in Southhampton.

## Feature Engineering

1. Cabin: simplify cavin by evaluatin if cabin letter or the purchase of tickets across multiple cabins impacted survival

2. Tickets - Do different ticket types impact survival rates?

3. Title - Does a persons title name relate to survival rate?

### Cabin


In [ ]:
#this formula basically assigns a number based on how many people had multiple cabins in the ship
df_cat.Cabin
training['cabin_multiple'] = training.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))

#now we show those values
training['cabin_multiple'].value_counts()

In [ ]:
a = 180+16+2
print('One cabin reserved: ', 180/a)
print('Two cabins reserveds: ', 16/a)
print('Three cabins reserveds: ', 2/a)


We conclud that over 90% of people had one cabin reserved. And that less than 1% had three.

We will look now if that variables has influence on survival rates.

In [ ]:
pd.pivot_table(training, index = 'Survived', columns='cabin_multiple', values='Ticket', aggfunc = 'count')

In [ ]:
print('People with 1 cabin that survived: ', 122/(58+122))
print('People with 2 cabins that survived: ', 9/(9+7))
print('People with 3 cabins that survivd: ', 3/(3+3))

This insight says at first hand that people with less cabins have more survival rates. Though, we have to say that the amount of people with 2 and 3 cabins is very low which raises the alarm that this insight might not be trustworthy.

Now we'll see if the letter of the cabin they where affects their survival rate. You would expect cabins with same letters to be on the same part of the ship, therefore it may be good information.

In [ ]:
#create categories based on the cabin letter (n= null)
training['cabin_adv'] = training.Cabin.apply(lambda x: str(x)[0])

In [ ]:
#comparting survival rates by cabin 
print(training.cabin_adv.value_counts())
pd.pivot_table(training, index = 'Survived', columns = 'cabin_adv', values = 'Ticket', aggfunc = 'count')

For all the cabin letter, except A, the survival rate was over 50%. But for people who don't actually have a value in it, therefore, it is null, the survival rate was less than 30%. This might be good information.

### Tickets

We want to understand ticket values better

We'll do numeric vs non-numeric

In [ ]:
training['numeric_ticket'] = training.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
training['ticket_letters'] = training.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1])>0 else 0)

#visualize it
training['numeric_ticket'].value_counts()

#if they have a number it's a 1, otherwise it's a 0

Let's look at all rows in dataframe through scrolling.

In [ ]:
pd.set_option('max_rows', None)
training['ticket_letters'].value_counts()

Explore impact of numeric ticket in survival rate

In [ ]:
pd.pivot_table(training, index = 'Survived', columns='numeric_ticket', values='Ticket', aggfunc = 'count')

In [ ]:
print('Non-numeric ticket survival rate: ', 88/(88+142))
print('Numeric ticket survival rate: ', 254/(254+407))

This rates are pretty much the same so I don't think this information will be helpful for predicting survival rates.

In [ ]:
#now with ticker letters
pd.pivot_table(training, index = 'Survived', columns='ticket_letters', values='Ticket', aggfunc = 'count')

Again,nothing really relevant.

### Person title

In [ ]:
training.Name.head(50)
training['name_title'] = training.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

#visualizing
training['name_title'].value_counts()

In [ ]:
pd.pivot_table(training, index = 'Survived', columns='name_title', values='Ticket', aggfunc = 'count')

 ## Data Preprocessing for Model
 
 1. Drop null values from Embarked (only 2)
 
 2. Include only only relevant variables (exclude variables that are now giving us much information)
 
 3. Make a categorical transform on all data. 
 
 4. Input data with mean for date and age (Should also experiment with median)
 
 5. Normalized fare using logarithm to give more semblance of a normal distribution
 
 6. Scaled data 0-1 sith standard sacler

#### Drop null values

In [ ]:
#Create categorical variables we did above for both training and test sets

all_data['cabin_multiple'] = all_data.Cabin.apply(lambda x: 0 if pd.isna(x) else len(x.split(' ')))
all_data['cabin_adv'] = all_data.Cabin.apply(lambda x: str(x)[0])
all_data['numeric_ticket'] = all_data.Ticket.apply(lambda x: 1 if x.isnumeric() else 0)
all_data['ticket_letters'] = all_data.Ticket.apply(lambda x: ''.join(x.split(' ')[:-1]).replace('.','').replace('/','').lower() if len(x.split(' ')[:-1])>0 else 0)
all_data['name_title'] = all_data.Name.apply(lambda x: x.split(',')[1].split('.')[0].strip())

In [ ]:
#inpute nulls for continuous data
all_data.Age = all_data.Age.fillna(training.Age.mean())
all_data.Fare = all_data.Fare.fillna(training.Fare.mean())

In [ ]:
#drop null embarked rows. Only 2 instances of this in training and 0 in test
all_data.dropna(subset=['Embarked'], inplace = True)

In [ ]:
#tried log norm of sibsp
all_data['norm_sibsp'] = np.log(all_data.SibSp+1)
all_data['norm_sibsp'].hist()

In [ ]:
#log norm of fare
all_data['norm_fare'] = np.log(all_data.Fare+1)
all_data['norm_fare'].hist()

In [ ]:
#converted fare to category for pd.get_dummies()
all_data.Pclass = all_data.Pclass.astype(str)

In [ ]:
#created dummy variables from categories
all_dummies = pd.get_dummies(all_data[['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'norm_fare', 'Embarked', 'cabin_adv', 'cabin_multiple', 'numeric_ticket', 'name_title', 'train_test']])

In [ ]:
#Split to train test again
X_train = all_dummies[all_dummies.train_test == 1].drop(['train_test'], axis=1)
X_test = all_dummies[all_dummies.train_test == 0].drop(['train_test'], axis=1)

In [ ]:
y_train = all_data[all_data.train_test==1].Survived
y_train.shape

In [ ]:
#scale data
from sklearn.preprocessing import StandardScaler
scale = StandardScaler()
all_dummies_scaled = all_dummies.copy()
all_dummies_scaled[['Age', 'SibSp', 'Parch', 'norm_fare']] = scale.fit_transform(all_dummies_scaled[['Age', 'SibSp', 'Parch', 'norm_fare']])
all_dummies_scaled

X_train_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 1].drop(['train_test'], axis = 1)
X_test_scaled = all_dummies_scaled[all_dummies_scaled.train_test == 0].drop(['train_test'], axis= 1)

y_train = all_data[all_data.train_test ==1].Survived

## Model building

First we are going to test multiple models to see which of them are more accurate for this type of prediction.

In [ ]:
#importing models
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

#### GausinanNB

In [ ]:
gnb = GaussianNB()
cv = cross_val_score(gnb,X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### Logistic regression

In [ ]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr, X_train, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

In [ ]:
lr = LogisticRegression(max_iter = 2000)
cv = cross_val_score(lr, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### Tree

In [ ]:
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt, X_train, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

In [ ]:
dt = tree.DecisionTreeClassifier(random_state = 1)
cv = cross_val_score(dt, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### K Nearest Neighbors

In [ ]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn, X_train, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

In [ ]:
knn = KNeighborsClassifier()
cv = cross_val_score(knn, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### Random forest

In [ ]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf, X_train, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

In [ ]:
rf = RandomForestClassifier(random_state = 1)
cv = cross_val_score(rf, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### SVC

In [ ]:
svc = SVC(probability = True)
cv = cross_val_score(rf, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### XGBoost

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 1)
cv = cross_val_score(rf, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

#### We now are going to use the voting classifier

This does a hard voting between all the classifiers. 

In [ ]:
from sklearn.ensemble import VotingClassifier
voting_clf = VotingClassifier(estimators = [('lr', lr), ('knn', knn), ('rf', rf), ('gnb', gnb), ('svc', svc), ('xgb', xgb)], voting = 'soft')

In [ ]:
#cv = cross_val_score(voting_clf, X_train_scaled, y_train, cv=5)
print(cv)
print('The cross_val_score is: ', cv.mean())

In [ ]:
#voting_clf.fit(X_train_scaled, y_train)
#y_hat_base_vc = voting_clf.predict(X_test_scaled).astype(int)
basic_submission = {'PassengerId': test.PassengerId, 'Survived': y_hat_base_vc}
base_submission = pd.DataFrame(data=basic_submission)
base_submission.to_csv('base_submission.csv', index=False)